In [ ]:
# import libraries
import os
import pandas as pd
from pandas import DataFrame
import numpy as np
from sklearn.preprocessing import MinMaxScaler
#from sklearn.externals import joblib
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
%matplotlib inline

from numpy.random import seed

#tf.logging.set_verbosity(tf.logging.ERROR)
from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, precision_recall_curve, cohen_kappa_score, fbeta_score
from sklearn.metrics import recall_score, classification_report, auc, roc_curve, log_loss
LABELS = ["Normal","FMA"]

#set up graphic style in this case I am using the color scheme from xkcd.com
from pylab import rcParams
rcParams['figure.figsize'] = 14, 8.7 # Golden Mean
LABELS = ["Normal","Fraud"]
col_list = ["cerulean","scarlet"]# https://xkcd.com/color/rgb/
sns.set(style='white', font_scale=1.75, palette=sns.xkcd_palette(col_list))

In [ ]:
import time
import joblib

In [ ]:
# set random seed
import tensorflow as tf
tf.random.set_seed(123)

In [ ]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

### **Cargar training y testing**

In [ ]:
# Cargar training y testing
# Read in data and display first 5 rows
test = pd.read_csv("Dataset_Test.csv")
# validation
train = pd.read_csv("Dataset_Training.csv")
print('The shape of our train is:', train.shape)
#test.head(5)
train.head(5)

## Seleccionar FMA en el test

In [ ]:
train.describe()

In [ ]:
train = train[train['FMA'] == 0] # Seleccion de datos

In [ ]:
train.head()

In [ ]:
split = 1460 # Validation
validation = train[-split:]
train = train[:-split]

In [ ]:
# The last element contains the labels
# Convertir Series to DataFrame (.to_frame())
train_fma = train.iloc[:, -1]
validation_fma = validation.iloc[:, -1]
test_fma = test.iloc[:, -1]

# Columnas
train_fma.columns = ['train_fma']
validation_fma.columns = ['validation_fma']
test_fma.columns = ['test_fma']

In [ ]:
test_fma.head()

https://www.tensorflow.org/guide/keras/train_and_evaluate#automatically_setting_apart_a_validation_holdout_set

### **Correlation**

In [ ]:
# test.corr()

In [ ]:
#train.corr()

In [ ]:
#plt.matshow(train.corr())
#plt.show()

### **Actualizar Datos**

In [ ]:
# Eliminar Campos
#test = test.drop(['temp_avg','temp_min', 'atmos_pres_min', 'wd', 'atmos_pres_max','atmos_pres_avg', 'rh','ceil_hgt', 'visibility', 'FMA'], axis=1) 
# ceil_hgt # visibility ,'temp_max','temp_min'
test = test.drop(['atmos_pres_min', 'atmos_pres_max','ceil_hgt', 'visibility', 'FMA'], axis=1)

In [ ]:
# Eliminar Campos
#train = train.drop(['temp_avg','temp_min', 'atmos_pres_min', 'wd', 'atmos_pres_max','atmos_pres_avg', 'rh','ceil_hgt', 'visibility', 'FMA'], axis=1)
train = train.drop(['atmos_pres_min', 'atmos_pres_max','ceil_hgt', 'visibility', 'FMA'], axis=1)

In [ ]:
# Eliminar Campos
#validation = validation.drop(['temp_avg','temp_min', 'atmos_pres_min', 'wd', 'atmos_pres_max','atmos_pres_avg', 'rh','ceil_hgt', 'visibility', 'FMA'], axis=1)
validation = validation.drop(['atmos_pres_min', 'atmos_pres_max','ceil_hgt', 'visibility', 'FMA'], axis=1)

In [ ]:
# Descriptive statistics for each column
#test.describe()

In [ ]:
print("Training dataset shape:", train.shape)
print("Test dataset shape:", validation.shape)
print("Test dataset shape:", test.shape)

In [ ]:
#train.corr()

In [ ]:
#test.corr()

In [ ]:
#plt.matshow(train.corr())
#plt.show()

In [ ]:
#fig, ax = plt.subplots(figsize=(14, 6), dpi=80)
#ax.plot(train['temp_max'], label='temp_max',  color='blue', animated = False,
#        linestyle='None', marker='*', linewidth=0.5)
#ax.plot(train['temp_avg'], label='temp_avg', color='red', animated = True, linewidth=1)
#plt.legend(loc='lower left')
#ax.set_title('Testing de lectura de Variables', fontsize=16)
#plt.show()

In [ ]:
#fig, ax = plt.subplots(figsize=(14, 6), dpi=80)
#ax.plot(train['ws'], label='ws',  color='blue', animated = False,
#        linestyle='None', marker='.', linewidth=0.5)
#ax.plot(train['temp_avg'], label='temp_avg', color='red', animated = True, linewidth=1)
#plt.legend(loc='lower left')
#ax.set_title('Testing de lectura de Variables', fontsize=16)
#plt.show()

## Normalize the data or Standardize the data ?

In [ ]:
# normalize the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(train)
X_test = scaler.transform(test)
X_validation = scaler.transform(validation)
scaler_filename = "scaler_data"
joblib.dump(scaler, scaler_filename)

In [ ]:
# reshape inputs for LSTM [samples, timesteps, features]
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1]) # X_train 
print("Training data shape:", X_train.shape)
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])     # X_test 
print("Test data shape:", X_test.shape)
X_validation = X_validation.reshape(X_validation.shape[0], 1, X_validation.shape[1])  #X_validation
print("Test data shape:", X_validation.shape)

In [ ]:
# define the autoencoder network model
def autoencoder_model(X):
    inputs = Input(shape=(X.shape[1], X.shape[2]))
    L1 = LSTM(32, activation='relu', return_sequences=True,  #16
              kernel_regularizer=regularizers.l2(0.00))(inputs)
    L2 = LSTM(4, activation='relu', return_sequences=False)(L1)
    L3 = RepeatVector(X.shape[1])(L2)
    L4 = LSTM(4, activation='relu', return_sequences=True)(L3)
    L5 = LSTM(32, activation='relu', return_sequences=True)(L4) #16
    output = TimeDistributed(Dense(X.shape[2]))(L5)    
    model = Model(inputs=inputs, outputs=output)
    return model

In [ ]:
# create the autoencoder model
model = autoencoder_model(X_train)
model.compile(optimizer='adamax', loss='mae', metrics=['mae', 'mse', 'mape', 'msle', 
                                                     'cosine_proximity']) #Error absoluto medio (mae), MeanSquaredError mse
model.summary()

In [ ]:
# collect data across multiple repeats
dtrain = DataFrame()
val = DataFrame()
# fit the model to the data
for i in range(2):
    print("Fit model on training data...")
    start = time.time()
    nb_epochs = 200 # 100
    batch_size = 32 # 10 # 32 casi por defecto...
    
    history = model.fit(X_train, X_train, epochs=nb_epochs, batch_size=batch_size,
                        validation_data=(X_validation, X_validation), verbose= 0).history # we use 5% of the training data for validation 
                                                                  # after each epoch (validation_split = 0.05)    # validation_data=X_validation
                        # validation_data=(X_validation, X_validation)
    end = time.time()
    # story history
    dtrain[str(i)] = history['loss']
    val[str(i)] = history['val_loss']
    print("Time to training model: ", end='')
    print(end - start)

In [ ]:
#X_train.shape

In [ ]:
# Grafo que evalua las distintas iteraciones
#print(dtrain.describe())
#dtrain.boxplot()
#plt.show()

In [ ]:
# print the best MSE reached on the test set
print("Best MSE on Validation Set =", max(history['val_mae']))

In [ ]:
#print("#########################################################")
loss = model.evaluate(X_train, X_train, verbose=0)
print(loss)

In [ ]:
#print(model.metrics_names)

In [ ]:
loss = model.evaluate(X_test, X_test, verbose=0)
#print(loss)

In [ ]:
#print('loss', loss[0])
#print('mae', loss[1])
#print('mse', loss[2])
#print('mape', loss[3])
#print('msle', loss[4])
#print('cosine_proximity', loss[5])

In [ ]:
# Mejor !!!
for name, value in zip(model.metrics_names, loss):
    print(name, value)

In [ ]:
# plot the training losses
fig, ax = plt.subplots(figsize=(14, 6), dpi=300)
ax.plot(history['loss'], 'b', label='Train', linewidth=2)
ax.plot(history['val_loss'], 'r', label='Validation', linewidth=2)
ax.set_title('Model loss', fontsize=16)
ax.set_ylabel('Loss (mae)')
ax.set_xlabel('Epoch')
ax.legend(loc='upper right')
plt.show()

In [ ]:
# plot the training losses
fig, ax = plt.subplots(figsize=(14, 6), dpi=80)
ax.plot(history['mse'], 'r', label='Train', linewidth=2)
ax.plot(history['val_mse'], 'g', label='Validation', linewidth=2)
ax.set_title('Training Mean Squared Error vs Validation Mean Squared Error', fontsize=16)
ax.set_ylabel('mse')
ax.set_xlabel('Epoch')
ax.legend(loc='upper right')
plt.show()

In [ ]:
# plot metrics
plt.plot(history['mse'],'r--,', label='mse')
plt.plot(history['mae'],'g', label='mae')
# plt.plot(history['mape'],'y', label='mape')
# plt.plot(history['cosine_proximity'], 'b', label='cosine_proximity')
plt.legend(loc='upper right')
plt.show()

## Distribution of Loss Function

In [ ]:
# plot the loss distribution of the training set
X_pred = model.predict(X_train)
X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])
X_pred = pd.DataFrame(X_pred, columns=train.columns)
X_pred.index = train.index

scored = pd.DataFrame(index=train.index)
Xtrain = X_train.reshape(X_train.shape[0], X_train.shape[2])
scored['Loss_mae'] = np.mean(np.abs(X_pred-Xtrain), axis = 1)
plt.figure(figsize=(16,9), dpi= 300)
plt.title('Loss Distribution', fontsize=16)
# Add a vertical line, here we set the style in the function call
plt.axvline(0.06, ls='--', color='r')
plt.annotate('Threshold = 0.06',
            xy=(0.06, 30), xycoords='data', fontsize=14,
            horizontalalignment='center', verticalalignment='bottom')
sns.distplot(scored['Loss_mae'], bins = 40, kde= True, color = 'blue');
plt.xlim([0.0,.4])

In [ ]:
#X_train.shape

In [ ]:
# calculate the loss on the test set
X_pred = model.predict(X_test)
X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])
X_pred = pd.DataFrame(X_pred, columns=test.columns)
X_pred.index = test.index

scored = pd.DataFrame(index=test.index)
Xtest = X_test.reshape(X_test.shape[0], X_test.shape[2])
scored['Loss_mae'] = np.mean(np.abs(X_pred-Xtest), axis = 1)
scored['Threshold'] = 0.025 # 0.275
scored['Anomaly'] = scored['Loss_mae'] > scored['Threshold']
#scored.head()

In [ ]:
scored.info()

In [ ]:
#scored.to_csv('FMA.csv', sep=',')

In [ ]:
data_filter = scored[scored['Anomaly'] == True]
#data_filter.head(1000)

In [ ]:
#data_filter.info()

In [ ]:
# Guardar los datos los FMA
#data_filter.to_csv('FMA.csv', sep=';')

In [ ]:
# calculate the same metrics for the training set 
# and merge all data in a single dataframe for plotting
X_pred_train = model.predict(X_train)
X_pred_train = X_pred_train.reshape(X_pred_train.shape[0], X_pred_train.shape[2])
X_pred_train = pd.DataFrame(X_pred_train, columns=train.columns)
X_pred_train.index = train.index

scored_train = pd.DataFrame(index=train.index)
scored_train['Loss_mae'] = np.mean(np.abs(X_pred_train-Xtrain), axis = 1)
scored_train['Threshold'] = 0.025 #0.275
scored_train['Anomaly'] = scored_train['Loss_mae'] > scored_train['Threshold']
scored = pd.concat([scored_train, scored])

Having calculated the loss distribution and the anomaly threshold, we can visualize the model output in the time leading up to the bearing failure.

In [ ]:
# plot bearing failure time plot
scored.plot(logy=True,  figsize=(16,9), ylim=[1e-2,1e2], color=['blue','red'])

## Determinación de threshold con (max)

In [ ]:
# Get train MAE loss.
x_train_pred = model.predict(X_train)
train_mae_loss = np.mean(np.abs(x_train_pred - X_train), axis=1)
plt.figure(figsize=(16,9), dpi= 300)
plt.hist(train_mae_loss, bins= 40)
plt.xlabel("Train MAE loss")
plt.ylabel("No of samples")
plt.xlim([0.0,.25])
plt.show()

In [ ]:
# Get reconstruction loss threshold.
threshold = np.max(train_mae_loss)
print("Reconstruction error threshold: ", threshold)


In [ ]:
#print("TF version:" + tf.__version__)
#print(tf.config.list_physical_devices('GPU'))

## Determinación de threshold con (mean + std)

In [ ]:
# Forma tonta de hacer lo mismo
reconstructions = model.predict(X_train)
train_loss = np.mean(np.abs(reconstructions - X_train), axis=1)
threshold = np.mean(train_loss) + np.std(train_loss)
print("Threshold: ", threshold)

In [ ]:
def print_stats(ytest, ypred):
    print("Accuracy: {:.5f}, Cohen's Kappa Score: {:.5f}".format(
        accuracy_score(ytest, ypred), 
        cohen_kappa_score(ytest, ypred, weights="quadratic")))
    ll = log_loss(ytest, ypred)
    print("Log Loss: {}".format(ll))
    print(' ')
    print("Confusion Matrix:")
    print(confusion_matrix(ytest, ypred))
    print("Classification Report:")
    print(classification_report(ytest, ypred))

# Version Dago .02
## Determinación de Clasificación. Test
## Análisis más aceptado!!!

In [ ]:
threshold_fixed = 0.01 #0.1
# calculate the loss on the test set
X_pred = model.predict(X_test)
X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])
X_pred = pd.DataFrame(X_pred, columns=test.columns)
X_pred.index = test.index

In [ ]:
error_df = pd.DataFrame(index=test.index)
Xtest = X_test.reshape(X_test.shape[0], X_test.shape[2])
error_df['Reconstruction_error'] = np.mean(np.power(X_pred-Xtest, 2), axis = 1)
error_df['True_class'] = test_fma
error_df.head()

In [ ]:
precision_rt, recall_rt, threshold_rt = precision_recall_curve(error_df.True_class, error_df.Reconstruction_error)
plt.plot(threshold_rt, precision_rt[1:], label="Precision",linewidth=2)
plt.plot(threshold_rt, recall_rt[1:], label="Recall",linewidth=2)
plt.title('Precision and recall for different threshold values')
plt.xlabel('Threshold')
plt.ylabel('Precision/Recall')
#plt.xlim([0.0,0.2]) # Permite gestionar la figura
plt.legend()
plt.show()

In [ ]:
print(threshold_rt)

## ROC Curve Check

In [ ]:
false_pos_rate, true_pos_rate, thresholds = roc_curve(error_df.True_class, error_df.Reconstruction_error)
roc_auc = auc(false_pos_rate, true_pos_rate,)

plt.plot(false_pos_rate, true_pos_rate, linewidth = 1, label='AUC = %0.3f'% roc_auc)
plt.plot([0,1],[0,1], linewidth = 3)

plt.xlim([-0.01, 1])
plt.ylim([0, 1.01])
plt.legend(loc='lower right')
plt.title('Receiver operating characteristic curve (ROC)')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

## Recall vs. Precision Thresholding

In [ ]:
precision_rt, recall_rt, threshold_rt = precision_recall_curve(error_df.True_class, error_df.Reconstruction_error)
plt.plot(recall_rt, precision_rt, linewidth = 1, label='Precision-Recall curve')
plt.legend(loc='upper right')
plt.title('Recall vs Precision')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

### Reconstruction Error vs Threshold Check. Importante threshold_fixed

In [ ]:
#threshold_fixed = 0.07 #0.1
groups = error_df.groupby('True_class')
fig, ax = plt.subplots()

for name, group in groups:
    ax.plot(group.index, group.Reconstruction_error, marker='o', ms=3.5, linestyle='',
            label= "FMA" if name == 1 else "Normal")
ax.hlines(threshold_fixed, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='Threshold')
ax.legend()
plt.title("Reconstruction error for different classes")
plt.ylabel("Reconstruction error")
plt.xlabel("Data point index")
plt.show(); 

### Confusion Matrix

In [ ]:
pred_y = [1 if e > threshold_fixed else 0 for e in error_df.Reconstruction_error.values]
conf_matrix = confusion_matrix(error_df.True_class, pred_y)

plt.figure(figsize=(12, 12))
sns.heatmap(conf_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d");
plt.title("Confusion matrix")
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()

In [ ]:
# Forma tonta de hacer lo mismo
reconstructions = model.predict(X_test)
test_loss = np.mean(np.abs(reconstructions - X_test), axis=1)
threshold = np.mean(test_loss) + np.std(test_loss)
print("Threshold: ", threshold)

In [ ]:
# error_df.True_class

In [ ]:
print_stats(error_df.True_class, pred_y)

In [ ]:
p = precision_score(error_df.True_class, pred_y)
r = recall_score(error_df.True_class, pred_y)
f = fbeta_score(error_df.True_class, pred_y, beta= 0.5)
print('Result: p=%.3f, r=%.3f, f=%.3f' % (p, r, f))

## Reconstruction error - Test set

In [ ]:
#plt.figure(figsize=(10,6))
sns.kdeplot(error_df.Reconstruction_error[error_df.True_class==0], label='Normal', shade=True, clip=(0,10))
sns.kdeplot(error_df.Reconstruction_error[error_df.True_class==1], label='FMA', shade=True, clip=(0,10))
# Add a vertical line, here we set the style in the function call
#plt.axvline(0.01, ls='--', color='r')
#plt.annotate('Threshold = 0.01',
#            xy=(0.01, 250), xycoords='data', fontsize=14,
#            horizontalalignment='center', verticalalignment='bottom')
plt.xlabel('Reconstruction error');
plt.title('Reconstruction error - Test set')

In [ ]:
pred_y = [1 if e > threshold_fixed else 0 for e in error_df.Reconstruction_error.values]
conf_matrix = confusion_matrix(error_df.True_class, pred_y)


In [ ]:
# threshold
for threshold_fixed in [0.01, 0.02, 0.04, 0.05]:
    pred_y = [1 if e > threshold_fixed else 0 for e in error_df.Reconstruction_error.values]
    conf_matrix = confusion_matrix(error_df.True_class, pred_y)
    print(conf_matrix)
    p = precision_score(error_df.True_class, pred_y)
    r = recall_score(error_df.True_class, pred_y)
    f = fbeta_score(error_df.True_class, pred_y, beta= 0.5)
    print('Result: p=%.3f, r=%.3f, f=%.3f' % (p, r, f))
    print_stats(error_df.True_class, pred_y)

## Hyperparameter